In [21]:
import numpy as np
import pandas as pd
import nltk
import tensorflow
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer   # stemming 
from nltk.stem import WordNetLemmatizer  # Lemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer  # transform our data into num
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import LSTM,Dense , SimpleRNN
print('libraries imported ')


libraries imported 


In [2]:
#reading data from text file 
train_data = open("Data/train.txt").readlines()
test_data = open("Data/test.txt").readlines()
val_data = open("Data/val.txt").readlines()

In [3]:
len(train_data)

16000

In [4]:
len(test_data)


2000

In [5]:
len(val_data)


2000

In [6]:
ls1 = [1,2,3]
ls2 = [4,5,6]
ls3 = [7,8,9]
ls1 + ls2 + ls3 

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
total_message = train_data + test_data + val_data 
len(total_message)

20000

In [8]:
total_message[10]

'i feel like i have to make the suffering i m seeing mean something;sadness\n'

In [4]:
x = []  # message 
y = []  # lables 
for item in total_message:
    text,label = item.split(';')
    label = label.replace('\n','')
    x.append(text)
    y.append(label)

# unique lables 

In [20]:
len(set(y))

### 
# joy         6761
# sadness     5797
# anger       2709
# fear        2373
# love        1641
# surprise     719

6

In [12]:
y[:5]

['sadness', 'sadness', 'anger', 'love', 'anger']

In [13]:
x[:5]

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy']

In [5]:
stemming = PorterStemmer()
def text_cleaning(ls_sentences):
    clean_message_ls = []
    for sentece in ls_sentences:
        lower_sentence = sentece.lower() # lower case
        ls_of_words = nltk.word_tokenize(lower_sentence)
        ls_of_words_removed_stopwords = [i for i in ls_of_words if i not in stopwords.words('english')]  # stopwords removing
        
        stemmed_word_ls = [stemming.stem(word)  for word in ls_of_words_removed_stopwords]  # stemming apply
        cleaned_message = " ".join(stemmed_word_ls) # joining and making a sentence
        clean_message_ls.append(cleaned_message)
    return clean_message_ls
        


cleaned_message_ls = text_cleaning(x)

In [45]:
len(cleaned_message_ls)  , len(x)

(20000, 20000)

In [46]:
x[25]

'i still love my so and wish the best for him i can no longer tolerate the effect that bm has on our lives and the fact that is has turned my so into a bitter angry person who is not always particularly kind to the people around him when he is feeling stressed'

In [47]:
cleaned_message_ls[25]

'still love wish best longer toler effect bm live fact turn bitter angri person alway particularli kind peopl around feel stress'

In [6]:
tokenizer = Tokenizer(oov_token="<nothing>")

In [7]:
tokenizer.fit_on_texts(cleaned_message_ls)

In [51]:
tokenizer.document_count

20000

In [64]:
# tokenizer.word_counts 

In [63]:
# tokenizer.word_index 

In [8]:
sequence = tokenizer.texts_to_sequences(cleaned_message_ls)

In [61]:
cleaned_message_ls[0]

'didnt feel humili'

In [60]:
sequence[0]

[61, 2, 522]

In [69]:
max(list(map(len,sequence)))

35

In [9]:
sequences  = pad_sequences(sequences=sequence , padding='post',maxlen=35)

In [10]:
sequences

array([[  61,    2,  522, ...,    0,    0,    0],
       [  10,    2,  419, ...,    0,    0,    0],
       [   4, 1230,  431, ...,    0,    0,    0],
       ...,
       [   2,  194,  157, ...,    0,    0,    0],
       [ 328,    2,  175, ...,    0,    0,    0],
       [   2,    3,  916, ...,    0,    0,    0]])

In [13]:

# label encoding 
# y1 = 0 1 2 3 4 5 
# y2 = one hot encoding 
# a    b    c     d    e    f     g 
# 1    0    0     0    0    0     0
# 0    0    1     0    0    0     0


# binary classification  
# multiclass 
label_dict = {
    'sadness':0,
    'anger':1,
    'love':2,
    'surprise':3,
    'fear':4,
    'joy':5
}



def label_encoder(labels):
    label = []
    for lab in labels:
        label.append(label_dict[lab]) # label_dict['sadness']
    label = np.array(label)
    return label

# label encoding  
label = label_encoder(y)


In [15]:
label

array([0, 0, 1, ..., 5, 5, 5])

In [ ]:
   # x , y 

In [18]:
x_train,x_test , y_train,y_test = train_test_split(sequences , label,test_size=0.2)

In [19]:
x_train.shape , y_train.shape

((16000, 35), (16000,))

In [20]:
x_test.shape , y_test.shape 

((4000, 35), (4000,))

In [ ]:
# model train 
# ML , ANN , CNN , RNN 
# ANN --> ML algorithms related like classification, regression 
# CNN --> when we have graphical data like image videos (computer vision)
# RNN --> text data (Natural language processing)

In [22]:
x_train.shape 

(16000, 35)

In [28]:
y_train

array([4, 5, 0, ..., 2, 0, 1])

In [26]:
model = Sequential()
model.add(SimpleRNN(32,input_shape=(35,1),return_sequences=False))  # hidden and input

model.add(Dense(6,activation='softmax'))  # output layer 



model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 32)                1088      
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 1286 (5.02 KB)
Trainable params: 1286 (5.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
label_dict

{'sadness': 0, 'anger': 1, 'love': 2, 'surprise': 3, 'fear': 4, 'joy': 5}

In [29]:
history = model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5


500/500 [==============================] - 9s 12ms/step - loss: 1.5881 - accuracy: 0.3264 - val_loss: 1.5856 - val_accuracy: 0.3410
Epoch 2/5
500/500 [==============================] - 6s 12ms/step - loss: 1.5745 - accuracy: 0.3338 - val_loss: 1.5862 - val_accuracy: 0.3392
Epoch 3/5
500/500 [==============================] - 6s 11ms/step - loss: 1.5743 - accuracy: 0.3349 - val_loss: 1.5855 - val_accuracy: 0.3392
Epoch 4/5
500/500 [==============================] - 5s 11ms/step - loss: 1.5740 - accuracy: 0.3352 - val_loss: 1.5866 - val_accuracy: 0.3397
Epoch 5/5
500/500 [==============================] - 5s 11ms/step - loss: 1.5734 - accuracy: 0.3339 - val_loss: 1.5842 - val_accuracy: 0.3390


In [ ]:





## compiling the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()